## Data Processing Stage-2 [Preparing Tefla ready data]

This Notebook runs after the DataProcessing-1.ipynb notebook. This notebook creates the csv file with consists of image name and its corresponding label. Then it prepares the data in the tefla ready format, which the adequate distribution of data between training, validation, and test for all the categories.

In [2]:
import os
import pandas as pd
import math
import numpy as np
from PIL import Image
import glob
os.getcwd()


'/home/ec2-user/src/tefla'

In [6]:

# script to create a tefla compatible data dir for training and validation data
sourceDir = '../../data/clothing/'
destDir = '../../data/clothing/koovs_tefla_ready/'
validation_data_percentage = 10
test_data_percentage = 10
categories = range(24)
def create_tefla_data(source_dir,destination_dir,validation_percentage,test_percentage,categories):
    if os.path.exists(destination_dir):
        shutil.rmtree(destination_dir)

    training_dir = destination_dir + 'training_224/'
    validation_dir = destination_dir + 'validation_224/'
    test_dir = destination_dir + 'test_224/'
    os.makedirs(training_dir)
    os.makedirs(validation_dir)
    os.makedirs(test_dir)

    labels = pd.read_csv( source_dir + 'all_koovs_new.csv')
    validation_dict = {}
    test_dict = {}

    for c in categories:
        validation_dict[c] = []

    for c in categories:
        test_dict[c] = []


    # test set creation
    test_set = []

    for i, rows in labels.iterrows():
        if test_dict.has_key(rows['labels']):
            test_dict[rows['labels']].append(rows['image'])

    for k in test_dict:
        np.random.seed(0)
        n = math.ceil(test_percentage * len(test_dict[k]) / 100.0)
        random_array = np.random.choice(test_dict[k],int(n))
        test_set = test_set + random_array.tolist()


    #validation set creation
    validation_set = []

    for i, rows in labels.iterrows():
        if rows['image'] not in test_set:
            if validation_dict.has_key(rows['labels']):
                validation_dict[rows['labels']].append(rows['image'])

    for l in validation_dict:
        np.random.seed(0)
        n = math.ceil(validation_percentage * len(validation_dict[l]) / 100.0)
        random_array = np.random.choice(validation_dict[l],int(n))
        validation_set = validation_set + random_array.tolist()

    training_images = []
    training_labels = []
    validation_images = []
    validation_labels =[]
    test_images = []
    test_labels = []

    for i, rows in labels.iterrows():
        new_name = rows['image']
        new_label = rows['labels']

        if rows['labels'] == 2 :
            new_label = 1

        if rows['image'] in test_set:
            test_images.append(new_name)
            test_labels.append(new_label)
            process_and_save_image(sourceDir + 'koovs/' + rows['image'] + ".jpg", test_dir + new_name + ".jpg");
        elif rows['image'] in validation_set:
            validation_images.append(new_name)
            validation_labels.append(new_label)
            process_and_save_image(sourceDir + 'koovs/' + rows['image'] + ".jpg", validation_dir + new_name + ".jpg")
        else:
            training_images.append(new_name)
            training_labels.append(new_label)
            process_and_save_image(sourceDir + 'koovs/' + rows['image'] + ".jpg", training_dir + new_name + ".jpg")

    header = ['image', 'label']

    # saving training csv
    training_out = np.column_stack((training_images, training_labels))
    training_out = np.row_stack((header, training_out))
    np.savetxt(destination_dir + 'training_labels.csv', training_out, delimiter=',', fmt='%s')

    # saving validation csv
    validation_out = np.column_stack((validation_images, validation_labels))
    validation_out = np.row_stack((header, validation_out))
    np.savetxt(destination_dir + 'validation_labels.csv', validation_out, delimiter=',', fmt='%s')

    # saving testing csv
    test_out = np.column_stack((test_images, test_labels))
    test_out = np.row_stack((header, test_out))
    np.savetxt(destination_dir + 'test_labels.csv', test_out, delimiter=',', fmt='%s')

def process_and_save_image(source_path,destination_path):
    img = resize(source_path, 224)
    img.save(destination_path, quality=97)

def resize(fname, target_size):
    # print('Processing image: %s' % fname)
    img = Image.open(fname)
    resized = img.resize([target_size, target_size])
    return resized

#calling method
create_tefla_data(sourceDir,destDir,validation_data_percentage,test_data_percentage,categories)



In [ ]:
!python -m tefla.train --model examples/mnist/mnist_model.py.py --training_cnf examples/mnist/mnist_cnf.py --data_dir /home/ec2-user/data/clothing/koovs_tefla_ready/  


I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcublas.so.7.5 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcudnn.so.5 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcufft.so.7.5 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcuda.so.1 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcurand.so.7.5 locally
2017-05-06 12:06:37,397 - INFO - Using parallel training iterator
2017-05-06 12:06:37,923 - INFO - Config:
2017-05-06 12:06:37,925 - INFO - {'aug_params': {'allow_stretch': True,
                'do_flip': False,
                'rotation_range': (-5, 5),
                'shear_range': (0, 0),
                'translation_range': (-2, 2),
                'zoom_range': (0.9523809523809523, 1.05)},
 'batch_size_test': 128,
 'batch_size_train': 128,
 'classification': True,
 '

In [5]:
!python -m tefla.train --help


I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcublas.so.7.5 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcudnn.so.5 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcufft.so.7.5 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcuda.so.1 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcurand.so.7.5 locally
Usage: train.py [OPTIONS]

Options:
  --model TEXT                   Relative path to model.
  --training_cnf TEXT            Relative path to training config file.
  --data_dir TEXT                Path to training directory.
  --start_epoch INTEGER          Epoch number from which to resume training.
                                 [default: 1]
  --resume_lr TEXT               Learning rate for resumed training.
  --weights_from TEXT            Path to initial weights fil